# 타이타닉 생존 분석: 머신러닝 모델링

## 1. 데이터 준비 및 전처리

### 1.1 필요한 라이브러리 및 데이터 로드

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# 데이터 로드
df = pd.read_csv('../data/processed/titanic_processed.csv')

# 기본 정보 확인
print("데이터셋 기본 정보:")
print(df.info())

데이터셋 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   survived       891 non-null    int64  
 1   pclass         891 non-null    int64  
 2   sex            891 non-null    object 
 3   age            891 non-null    float64
 4   sibsp          891 non-null    int64  
 5   parch          891 non-null    int64  
 6   fare           891 non-null    float64
 7   embarked       891 non-null    object 
 8   who            891 non-null    object 
 9   adult_male     891 non-null    bool   
 10  deck           891 non-null    object 
 11  alone          891 non-null    bool   
 12  family_size    891 non-null    int64  
 13  family_type    891 non-null    object 
 14  age_group      891 non-null    object 
 15  fare_category  891 non-null    object 
dtypes: bool(2), float64(2), int64(5), object(7)
memory usage: 99.3+ KB
None


### 1.2 특성 선택 및 전처리

In [2]:
# 범주형 변수 인코딩
df_encoded = pd.get_dummies(df, columns=['sex', 'embarked', 'deck', 'age_group', 'fare_category'])

# 사용할 특성 선택
features = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'family_size'] + \
          [col for col in df_encoded.columns if 'sex_' in col or 
                                               'embarked_' in col or 
                                               'deck_' in col or 
                                               'age_group_' in col or 
                                               'fare_category_' in col]

X = df_encoded[features]
y = df_encoded['survived']

# 훈련/테스트 세트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n선택된 특성:", len(features))
print("\n처음 5개 특성:", features[:5])
print("\n데이터 shape:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


선택된 특성: 28

처음 5개 특성: ['pclass', 'age', 'sibsp', 'parch', 'fare']

데이터 shape:
X_train shape: (712, 28)
X_test shape: (179, 28)


## 2. 머신러닝 모델 구현 및 평가

### 2.1 기본 모델 구현 및 평가 함수

In [3]:
def evaluate_model(y_true, y_pred, model_name):
    """
    모델 성능 평가 및 결과 출력 함수
    """
    print(f"\n{model_name} 성능 평가:")
    print("정확도 (Accuracy):", accuracy_score(y_true, y_pred))
    print("정밀도 (Precision):", precision_score(y_true, y_pred))
    print("재현율 (Recall):", recall_score(y_true, y_pred))
    print("F1 점수:", f1_score(y_true, y_pred))
    print("\n혼동 행렬:")
    print(confusion_matrix(y_true, y_pred))
    print("\n분류 보고서:")
    print(classification_report(y_true, y_pred))

# 1. 로지스틱 회귀
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)
y_pred_log = log_reg.predict(X_test_scaled)
evaluate_model(y_test, y_pred_log, "로지스틱 회귀")

# 2. 의사결정나무
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_scaled, y_train)
y_pred_dt = dt.predict(X_test_scaled)
evaluate_model(y_test, y_pred_dt, "의사결정나무")

# 3. 랜덤 포레스트
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
evaluate_model(y_test, y_pred_rf, "랜덤 포레스트")

# 특성 중요도 시각화 (랜덤 포레스트 기준)
plt.figure(figsize=(12, 6))
importances = pd.Series(rf.feature_importances_, index=features)
importances = importances.sort_values(ascending=True)
importances.plot(kind='barh')
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()


로지스틱 회귀 성능 평가:
정확도 (Accuracy): 0.7877094972067039
정밀도 (Precision): 0.7368421052631579
재현율 (Recall): 0.7567567567567568
F1 점수: 0.7466666666666667

혼동 행렬:
[[85 20]
 [18 56]]

분류 보고서:
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       105
           1       0.74      0.76      0.75        74

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179


의사결정나무 성능 평가:
정확도 (Accuracy): 0.7653631284916201
정밀도 (Precision): 0.7
재현율 (Recall): 0.7567567567567568
F1 점수: 0.7272727272727273

혼동 행렬:
[[81 24]
 [18 56]]

분류 보고서:
              precision    recall  f1-score   support

           0       0.82      0.77      0.79       105
           1       0.70      0.76      0.73        74

    accuracy                           0.77       179
   macro avg       0.76      0.76      0.76       179
weighted avg       0.77      0.77      0.77       179

NameError: name 'plt' is not defined